In [1]:
!pip install pyspark
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=f83756e06f82dacaf9db9f895bee8bde22091e101edb1aa8e2f1b386072ecc8c
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Ge

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/transactions_train.csv', header=True, inferSchema=True)
transactions_rdd = df.rdd

In [4]:
transactions_rdd.first()

Row(_c0=0, Id=2987000, target=0, timestamp=86400, ammount=68.5, product_category='W', feat_card_1=13926, feat_card_2=None, feat_card_3=150.0, feat_card_4='discover', feat_card_5=142.0, feat_card_6='credit', addr1=315.0, addr2=87.0, dist1=19.0, dist2=None, payer_email_domain=None, receiver_email_domain=None, A1=1.0, A2=1.0, A3=0.0, A4=0.0, A5=0.0, A6=1.0, A7=0.0, A8=0.0, A9=1.0, A10=0.0, A11=2.0, A12=0.0, A13=1.0, A14=1.0, B1=14.0, B2=None, B3=13.0, B4=None, B5=None, B6=None, B7=None, B8=None, B9=None, B10=13.0, B11=13.0, B12=None, B13=None, B14=None, B15=0.0, C1='T', C2='T', C3='T', C4='M2', C5='F', C6='T', C7=None, C8=None, C9=None, D1=1.0, D2=1.0, D3=1.0, D4=1.0, D5=1.0, D6=1.0, D7=1.0, D8=1.0, D9=1.0, D10=0.0, D11=0.0, D12=1.0, D13=1.0, D14=1.0, D15=0.0, D16=0.0, D17=0.0, D18=0.0, D19=1.0, D20=1.0, D21=0.0, D22=0.0, D23=1.0, D24=1.0, D25=1.0, D26=1.0, D27=0.0, D28=0.0, D29=0.0, D30=0.0, D31=0.0, D32=0.0, D33=0.0, D34=0.0, D35=None, D36=None, D37=None, D38=None, D39=None, D40=None, D

Porcentaje de fraude

In [ ]:
transactions_rdd.map(lambda x: x.target).distinct().collect()

[0, 1]

In [ ]:
transactions_rdd.filter(lambda x: x.target == 1).count() * 100 / transactions_rdd.count()

3.502278074504093

Porcentaje de monto fraudulento respecto de los productos

In [ ]:
fraude = transactions_rdd.filter(lambda x: x.target == 1).map(lambda x: (x.product_category, x.ammount)).reduceByKey(lambda a,b: a + b)

In [ ]:
legal = transactions_rdd.filter(lambda x: x.target == 0).map(lambda x: (x.product_category, x.ammount)).reduceByKey(lambda a,b: a + b)

In [ ]:
producto_monto_f_l = fraude.fullOuterJoin(legal).collect()

In [ ]:
def porcentaje(lista):
  promedios = []
  for producto in lista:
    promedios.append((producto[0], producto[1][0] * 100 / (producto[1][0] + producto[1][1])))
  return promedios

porcentaje(producto_monto_f_l)

[('H', 10.087869182508362),
 ('R', 5.134079848889628),
 ('S', 7.983558441966235),
 ('C', 12.916996422843251),
 ('W', 3.0548043859574134)]

Fraude segun el tiempo, analizando cantidad y monto

In [5]:
fraude = transactions_rdd.filter(lambda x: x.target == 1).cache()

In [48]:
def segundos(numero):
  return numero - 86400

def analisis(numero, divisor):
  flotante = (numero - 86400) / divisor
  entero = (numero - 86400) // divisor
  if flotante > entero:
    return entero + 1
  else:
    return entero

def minutos(numero):
  return analisis(numero, 60)

def horas(numero):
  return analisis(numero, 3600)

def dias(numero):
  return analisis(numero, 86400)

def semanas(numero):
  return analisis(numero, 604800)

def meses(numero):
  return analisis(numero, 2419200)

Cantidad respecto del tiempo

Máxima cantidad respecto de segundos

In [57]:
(fraude.map(lambda x: segundos(x.timestamp)).reduce(lambda a,b: a if a > b else b), fraude.count())

(13064705, 17580)

Máxima cantidad respecto de minutos

In [50]:
fraude.map(lambda x: (minutos(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b)

(136426, 6)

Máxima cantidad respecto de horas

In [51]:
fraude.map(lambda x: (horas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b)

(1529, 42)

Cantidad respecto de días

In [52]:
sorted(fraude.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect())

[(1, 112),
 (2, 123),
 (3, 92),
 (4, 115),
 (5, 127),
 (6, 99),
 (7, 136),
 (8, 93),
 (9, 119),
 (10, 90),
 (11, 123),
 (12, 84),
 (13, 82),
 (14, 126),
 (15, 143),
 (16, 70),
 (17, 126),
 (18, 161),
 (19, 118),
 (20, 135),
 (21, 116),
 (22, 147),
 (23, 106),
 (24, 76),
 (25, 63),
 (26, 82),
 (27, 100),
 (28, 150),
 (29, 129),
 (30, 158),
 (31, 149),
 (32, 51),
 (33, 123),
 (34, 159),
 (35, 120),
 (36, 135),
 (37, 126),
 (38, 110),
 (39, 117),
 (40, 109),
 (41, 121),
 (42, 85),
 (43, 160),
 (44, 119),
 (45, 100),
 (46, 117),
 (47, 91),
 (48, 115),
 (49, 121),
 (50, 175),
 (51, 148),
 (52, 114),
 (53, 79),
 (54, 112),
 (55, 76),
 (56, 158),
 (57, 109),
 (58, 102),
 (59, 168),
 (60, 108),
 (61, 112),
 (62, 165),
 (63, 163),
 (64, 234),
 (65, 183),
 (66, 85),
 (67, 120),
 (68, 111),
 (69, 118),
 (70, 145),
 (71, 152),
 (72, 94),
 (73, 135),
 (74, 120),
 (75, 110),
 (76, 144),
 (77, 125),
 (78, 100),
 (79, 113),
 (80, 66),
 (81, 105),
 (82, 95),
 (83, 81),
 (84, 116),
 (85, 187),
 (86, 125

Máxima cantidad respecto de días

In [53]:
fraude.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b)

(64, 234)

Cantidad respecto de semanas

In [54]:
sorted(fraude.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect())

[(1, 804),
 (2, 717),
 (3, 869),
 (4, 724),
 (5, 889),
 (6, 803),
 (7, 823),
 (8, 862),
 (9, 927),
 (10, 996),
 (11, 880),
 (12, 676),
 (13, 912),
 (14, 884),
 (15, 718),
 (16, 866),
 (17, 1069),
 (18, 833),
 (19, 727),
 (20, 611),
 (21, 639),
 (22, 351)]

Máxima cantidad respecto de semanas

In [55]:
fraude.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b)

(17, 1069)

Cantidad respecto de meses

In [56]:
sorted(fraude.map(lambda x: (meses(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect())

[(1, 3114), (2, 3377), (3, 3479), (4, 3380), (5, 3240), (6, 990)]